# merge with director-portfolio-years

### education and awards [15 Jan]

In [22]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None #default='warn'
pd.set_option('display.max_columns', None) #show all columns

df_awards = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\NA - SMDEs Profile - Education & Achievements_cleaned.xlsx", sheet_name="Achievements")

# df1 = pd.read_csv(r"C:\Users\g21728\Desktop\Directors\Data\Hannes\director-year_marketcap_capiq_datastream_merge_120k.csv", decimal=",", sep=";")
df2 = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\NA - SMDEs Profile - Education & Achievements_cleaned.xlsx", sheet_name="Education")
#assign ivyleague dummy if director attended at least one ivy league uni
ivy_league = "|".join(["Brown University", "Columbia University", "Cornell University", "Dartmouth College", "Harvard", "University of Pennsylvania", "Princeton", "Yale"])
df2["IvyLeague"] = df2["Institution Name"].str.contains(ivy_league).astype("Int64") #True if institution name contains any of the list items. convert boolean to 1 and 0
df2 = df2.drop(["Institution Name"], axis=1)
#groupby phd and ivyleague per director
df_edu = df2.groupby(["DirectorID"], as_index=False).max() #each director has multiple schools. reduce to one observation per director and set to one if at least one PhD received and one ivy league attended

df_edu

,DirectorID,PhD,IvyLeague
0,216931,0,1
1,500100,0,0
2,500160,0,1
3,611920,0,0
4,611970,0,1
...,...,...,...
298288,207071812928,0,0
298289,207073412928,0,0
298290,207075412928,0,0
298291,207075512928,0,0


In [ ]:
#group by awards sum per director
df_aw = df_awards.groupby(["DirectorID"], as_index=False).sum()
df_aw

#outer merge: keep all Dir observations of both dfs
df_edu_aw = df_aw.merge(df_edu, how="outer", on=["DirectorID"])

df_edu_aw.rename(columns={"AwardReceived": "DIR_awards", "PhD": "DIR_PhD", "IvyLeague": "DIR_ivy"}, inplace=True)
print(df_edu_aw)

df_latest = pd.read_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Director-Firm-Years\0115director-firm-years_iota.csv", sep=";", decimal='.')
df_latest_edu_aw = df_latest.merge(df_edu_aw, how="left", on=["DirectorID"])

#set awards to 0 for nan obs
df_latest_edu_aw["DIR_awards"] = np.where(df_latest_edu_aw["DIR_awards"].isna() == True, 0, df_latest_edu_aw["DIR_awards"])

#drop duplicates
df_latest_edu_aw.drop_duplicates(subset=["DirectorID", "FirmID", "Year"], inplace=True)

df_latest_edu_aw.to_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Director-Firm-Years\0115bdirector-firm-years_iota.csv", sep=";", decimal='.', index=False)

#### sort / reorder columns

In [49]:
dfq = df_latest_edu_aw

sorted_columns = dfq.columns.to_list()[:20] + sorted(dfq.columns.to_list()[20:], key=str.casefold) #keep first 20 columns. sort all others alphabetically & ignore case

dfq2 = dfq.reindex(columns=sorted_columns)
dfq2.drop(columns=["FE_Director_ISIN", "FE_SIC2_Year"], inplace=True)

dfq2.to_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Director-Firm-Years\0115bdirector-firm-years_iota.csv", sep=";", decimal='.', index=False)

# +Characteristics

In [7]:
import pandas as pd
import numpy as np

df_c1 = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\Characteristics\clean\NA - SMDEs Profile - Characteristics - 1_clean.xlsx")
df_c2 = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\Characteristics\clean\NA - SMDEs Profile - Characteristics - 2_clean.xlsx")
df_c3 = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\Characteristics\clean\NA - SMDEs Profile - Characteristics - 3_clean.xlsx")
df_c4 = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\Characteristics\clean\NA - SMDEs Profile - Characteristics - 4_clean.xlsx")

#combine to one df
df_characteristics = pd.concat([df_c1, df_c2, df_c3, df_c4], ignore_index=True)
df_characteristics

,DirectorID,DOB,Gender,Nationality,Year
0,11858664468,1963,M,Canadian,2010
1,11858664468,1963,M,Canadian,2011
2,11858664468,1963,M,Canadian,2012
3,11858664468,1963,M,Canadian,2013
4,11858664468,1963,M,Canadian,2014
...,...,...,...,...,...
1716560,8829597641,1953-05-14 00:00:00,M,Polish,NaN
1716561,8829597641,1953-05-14 00:00:00,M,Polish,NaN
1716562,8829597641,1953-05-14 00:00:00,M,Polish,NaN
1716563,8829597641,1953-05-14 00:00:00,M,Polish,NaN


In [ ]:
df_dob = df_characteristics[["DirectorID", "DOB", "Gender"]]
df_dob["Gender"].value_counts()
df_dob["Gender"].replace({"M": 0, "F": 1}, inplace=True)
df_dob["DOB"].replace({"Unknown": np.nan, pd.NA: np.nan}, inplace=True)
df_gender = df_dob[df_dob["Gender"].isin([0, 1])]

df_dob.dtypes

df_dob["DOB"] = df_dob["DOB"].astype("string").str[:4].astype(int) #pd.na in column -> cant be converted to int?
df_age = df_characteristics[df_characteristics["DOB"] > 0][["DirectorID", "DOB"]].sort_values(by=["DirectorID", "DOB"], ascending=[True,True]).reset_index()

In [44]:
df_gender.drop_duplicates(subset=["DirectorID"], keep="first", inplace=True)
df_gender.drop(columns=["DOB"], inplace=True)
df_gender

,DirectorID,Gender
0,11858664468,0
9,11615744030,0
11,11358623522,0
13,103832312113,0
28,13389206620,0
...,...,...
1716532,8768536979,0
1716534,7948969080,0
1716539,81138110481,0
1716544,7982899383,0


In [ ]:
xyz = df_age.merge(df_characteristics, how="left", on=["DirectorID"]).sort_values(by=["DirectorID", "Year"], ascending=[True,True], ignore_index=True)

#export
xyz.to_csv(r"C:\Users\g21728\Desktop\director-year-complete.csv", sep=";", decimal='.', index=False)

# add network size
## with director-firm-years
4 org summaries: daraus directorid, year und networksize
mit unseren dta file mergen

### org summary

In [39]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'

# df_org1 = pd.read_excel(r"D:\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\NA - SMDEs Org Summary - 1.xlsx")
# df_org2 = pd.read_excel(r"D:\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\NA - SMDEs Org Summary - 2.xlsx")
# df_org3 = pd.read_excel(r"D:\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\NA - SMDEs Org Summary - 3.xlsx")
# df_org4 = pd.read_excel(r"D:\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\NA - SMDEs Org Summary - 4.xlsx")

#combine org summaries to one df
# df_combined = pd.concat([df_org1, df_org2, df_org3, df_org4], ignore_index=True)
#safe to pickle (feather .ftr doesnt work since some columns are stored as object types)
# df_combined.to_pickle(r"C:\Users\hannes\Desktop\director-firm-years-networksize.pkl")

df_summary = pd.read_pickle(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Raw Data\NA - SMDEs Org Summary.pkl")

#set first row as headers
df_summary.columns = df_summary.iloc[0]
df_summary = df_summary[1:]
df_summary
#delete headers from df2-4
df_summary.drop_duplicates(inplace=True)
#rename column names
df_summary.rename(columns={"DirectorID*": "DirectorID", "CompanyID*": "CompanyID"}, inplace=True)
#delete last remaining header
df_summary[df_summary["DirectorID"] == "DirectorID*"].index
df_summary.drop(499878, inplace=True)

#convert string to int
df_summary['DirectorID'] = pd.to_numeric(df_summary['DirectorID'], errors='ignore') 

pd.set_option('display.max_columns', None)
df_summary.sort_values(by=["DirectorID"], inplace=True, ignore_index=True)

df_summary.head()

,Annual Report Year,Country,CompanyID,Company Name,ISIN,Ticker,Index,Sector,"Director Type (ED , SD or SM)",DirectorID,Individual Name,Individual Role,Director Network Size,Time to Retirement,Time in Role,Time on Board,Time in Company,Total Number of Quoted Boards to Date,Total Number of Private Boards to Date,Total Number of Other Boards to Date,Total Number of Quoted Current Boards,Total Number of Private Current Boards,Total Number of Other Current Boards,Avg. Yrs on Other Quoted Boards,Age (Yrs),Number of Qualifications,Gender,Nationality Mix,Bonus/ (Bonus&Salary),Equity Linked/ Total,Performance/ Total,%Change from Last Period,Wealth Delta,Salary,Bonus,D.C Pension,Other,Total Salary+Bonus,Total Inc. D.C. Pension & Other,Shares,LTIPS(max),Vesting Date,Intrinsic Options (excercisable),Intrinsic Options (unexercisable),Estimated Options (exercisable),Estimated Options (unexcercisable),Share Price,Total Equity Linked Compensation,Total Annual Compensation,Shares,LTIPS(max),Intrinsic Option,Estimated Option,Liquid Wealth,Total Wealth,AUDIT COMMITTEE MEMBER (if yes then put M),REMUNERATION/ COMPENSATION COMMITTEE MEMBER,NOMINATION COMMITTEE MEMBER,AUDIT COMMITTEE SIZE,REMUNERATION/ COMPENSATION COMMITTEE SIZE,NOMINATION COMMITTEE SIZE
0,Dec 2001,United States,217149383,IDEXX LABORATORIES INC,US45168D1046,IDXX,"NASDAQ 100, S&P 500",Pharmaceuticals and Biotechnology,ED,216931,David Evans Shaw,Chairman/President/CEO,4313,14.3,13.2,18.9,18.9,1,6,n.a,1,1,n.a,0,50,3,M,American,0.36,0.7,n.a,23.38,452,500,280,n.a,5,780,785,n.a,n.a,2003-02-07 00:00:00,n.a,123,n.a,369,28.83,1846,2626,10357,n.a,18109,25792,28466,36149,NaN,NaN,NaN,NaN,NaN,NaN
1,Dec 2000,United States,217149383,IDEXX LABORATORIES INC,US45168D1046,IDXX,"NASDAQ 100, S&P 500",Pharmaceuticals and Biotechnology,ED,216931,David Evans Shaw,Chairman/President/CEO,4313,15.3,12.2,17.9,17.9,1,6,n.a,1,1,n.a,0,49,3,M,American,0.45,0.61,n.a,n.a,375,467,383,n.a,5,849,855,n.a,n.a,2001-02-04 00:00:00,n.a,93,n.a,268,24.31,1339,2189,8741,n.a,12339,23819,21080,32560,NaN,NaN,NaN,NaN,NaN,NaN
2,Dec 2000,United States,217149383,IDEXX LABORATORIES INC,US45168D1046,IDXX,"NASDAQ 100, S&P 500",Pharmaceuticals and Biotechnology,ED,216931,David Evans Shaw,Chairman/President/CEO,4313,15.3,12.2,17.9,17.9,1,6,n.a,1,1,n.a,0,49,3,M,American,0.45,0.61,n.a,n.a,375,467,383,n.a,5,849,855,n.a,n.a,2002-02-04 00:00:00,n.a,93,n.a,268,24.31,1339,2189,8741,n.a,12339,23819,21080,32560,NaN,NaN,NaN,NaN,NaN,NaN
3,Dec 2000,United States,217149383,IDEXX LABORATORIES INC,US45168D1046,IDXX,"NASDAQ 100, S&P 500",Pharmaceuticals and Biotechnology,ED,216931,David Evans Shaw,Chairman/President/CEO,4313,15.3,12.2,17.9,17.9,1,6,n.a,1,1,n.a,0,49,3,M,American,0.45,0.61,n.a,n.a,375,467,383,n.a,5,849,855,n.a,n.a,2004-02-04 00:00:00,n.a,93,n.a,268,24.31,1339,2189,8741,n.a,12339,23819,21080,32560,NaN,NaN,NaN,NaN,NaN,NaN
4,Dec 2001,United States,217149383,IDEXX LABORATORIES INC,US45168D1046,IDXX,"NASDAQ 100, S&P 500",Pharmaceuticals and Biotechnology,ED,216931,David Evans Shaw,Chairman/President/CEO,4313,14.3,13.2,18.9,18.9,1,6,n.a,1,1,n.a,0,50,3,M,American,0.36,0.7,n.a,23.38,452,500,280,n.a,5,780,785,n.a,n.a,2005-02-07 00:00:00,n.a,123,n.a,369,28.83,1846,2626,10357,n.a,18109,25792,28466,36149,NaN,NaN,NaN,NaN,NaN,NaN


#### CEO turnover

In [35]:
# df_summary[df_summary["Individual Role"].str.contains("CEO", case=True)]
df_summary[df_summary["Individual Role"] == "CEO"]
df_summary["Individual Role"].nunique()

11275

In [ ]:
# annual report year: current -> 2019; [:-4]
df_summary.iloc[:,0].replace({"Current": 2019}, inplace=True) #df_summary["Annual Report Year"]. doesnt work for some reason
df_summary["Annual Report Year"] = np.where(df_summary["Annual Report Year"] == 2019, 2019, df_summary["Annual Report Year"].str[4:])
df_summary["Annual Report Year"] = df_summary["Annual Report Year"]
df_summary[df_summary["Annual Report Year"] == 2019]

In [ ]:
#convert all DOB entries to year ints
df_characteristics['Annual Report Year'] = df_characteristics['Annual Report Year'].astype(str).str[0:4] #take first 4 chars of date of birth
# df_characteristics["DOB"] = df_characteristics["DOB"].astype("float").astype("Int64") #convert year to int

In [12]:
df_ceo = df_summary[["Annual Report Year", "CompanyID", "DirectorID", "Individual Role"]]
df_ceo[["Individual Role"].str.contains("ceo", case=False)]

AttributeError: 'list' object has no attribute 'str'

In [3]:
df_summary = df_summary.loc[:,~df_summary.columns.duplicated()] #apparently there are duplicates in the df, which have to be removed before filter can be used
df_summary_filter = df_summary.filter(["DirectorID", "Annual Report Year", "Director Network Size"])
df_networksize = df_summary_filter.groupby("DirectorID")["Director Network Size"].max().reset_index(name="Director Network Size")
df_networksize

,DirectorID,Director Network Size
0,216931,4313
1,500100,281
2,500160,4979
3,611920,1621
4,611970,284
...,...,...
348788,207075412928,127
348789,207075512928,1087
348790,207075912928,110
348791,207076812928,276


In [4]:
# df_stata = pd.read_stata(r"C:\Users\g21728\Desktop\Directors\Stata\Stata_Data_Director_02122021.dta") #same as df_director_year_merge_education_achievements_characteristics_committee
df_stata = pd.read_csv(r"C:\Users\hannes\Desktop\director-firm-year-v2.csv", sep=";", decimal='.') #see director-firm-years.ipynb output
df_stata

,DirectorID,FirmID,Year,Vorhanden,years_before_gain,years_after_loss,SoloObservation,ISIN First,ISIN Alternative,FIRM Board Independence,FIRM Total Assets,FIRM Growth Rate of Assets,FIRM Cash,FIRM Debt to Capital,FIRM Closely Held Shares,FIRM EPS,FIRM ROA,BetaAverage,HistoricVolaAverage,BetaTotal,VolaTotal,TobinsQAverage,ROAAverage,EPSAverage,BoardSizeAverage,BoardMeetingsAverage,AttendanceLevel,LitigationExpensesAverage,IndependenceLevelAverage,AccrualsChangeAverage,TotalAssetsAverage,GrowthRateAssetsAverage,ImpairmentIntangiblesAverage,CashAverage,ImpairmentGoodwillAverage,DebttoCapitalAverage,CloselyHeldSharesAverage,AmortIntangGoodwillAverage,AmortOfdefChargesAverages,AmortIntangAverage,CommonSharesOutstandingAverage,BriberyCorruptionandFraudControv,RESEARCHDEVELOPMENTAVERAGE,EarlyAuditorResignation,ManagementDepatureAverage,GoodwillWrittenOffAverage,EarningsRestatementAverage,GovernanceScoreAverage,BoardSizeAverage1,NEDBoardSizeAverage,BoardCompensationAverage,ReviewAverage,RatingBalanceAverage,RatingCultureOverallAverage,RatingCareerAverage,RatingCompAverage,RecommendsAverage,PositiveOutlookAverage,ApprovesofCEOAverage,HelpfulAverage,CurrentEmployeeAverage,FullTimeAverage,TenureAverage,MarketCapAverage,MarketCapDelta,marketcapdummy,marketcapdummy_log,PF_changed,avg_previous_currentyear,avg_current,delta_currentprevious,realtivfemarketcapDelta,marketcapdummy_ersteBeobachtungb,gain_dummy,loss_dummy,replacement_dummy,change_dummy,event_gain_dummy,event_loss_dummy,event_replace_dummy,event_change_dummy,avg_current_valued_in_prioryear,delta_old,PhD,IvyLeague,AwardReceived,AuditCommittee,CompensationCommittee,NominationCommittee,GovernanceCommittee,FinanceRiskCommittee,Member,Chairman,ED,CEO,CFO,NumberofDirs,Nationality,NewAge,GenderDummy1M0F,SoleDirectorship,lnAge,gain_loss_replacement,new_chairman_dummy
0,3791088892,6613355791,1999,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3791088892,6613355791,2000,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3791088892,6613355791,2001,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3791088892,6613355791,2002,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3791088892,6613355791,2003,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,9.568189,-22.36,-8.857,NaN,NaN,NaN,NaN,NaN,NaN,18070.0,109.73,NaN,8551.0,NaN,34.66,60.87,NaN,NaN,NaN,3307.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165354.0,NaN,0.0,0.0,NaN,NaN,37.3953,NaN,NaN,NaN,NaN,N

In [5]:
#merge left
df_director_firm_year_merge_network = df_stata.merge(df_networksize, how="left", on="DirectorID")
df_director_firm_year_merge_network

,DirectorID,FirmID,Year,Vorhanden,years_before_gain,years_after_loss,SoloObservation,ISIN First,ISIN Alternative,FIRM Board Independence,FIRM Total Assets,FIRM Growth Rate of Assets,FIRM Cash,FIRM Debt to Capital,FIRM Closely Held Shares,FIRM EPS,FIRM ROA,BetaAverage,HistoricVolaAverage,BetaTotal,VolaTotal,TobinsQAverage,ROAAverage,EPSAverage,BoardSizeAverage,BoardMeetingsAverage,AttendanceLevel,LitigationExpensesAverage,IndependenceLevelAverage,AccrualsChangeAverage,TotalAssetsAverage,GrowthRateAssetsAverage,ImpairmentIntangiblesAverage,CashAverage,ImpairmentGoodwillAverage,DebttoCapitalAverage,CloselyHeldSharesAverage,AmortIntangGoodwillAverage,AmortOfdefChargesAverages,AmortIntangAverage,CommonSharesOutstandingAverage,BriberyCorruptionandFraudControv,RESEARCHDEVELOPMENTAVERAGE,EarlyAuditorResignation,ManagementDepatureAverage,GoodwillWrittenOffAverage,EarningsRestatementAverage,GovernanceScoreAverage,BoardSizeAverage1,NEDBoardSizeAverage,BoardCompensationAverage,ReviewAverage,RatingBalanceAverage,RatingCultureOverallAverage,RatingCareerAverage,RatingCompAverage,RecommendsAverage,PositiveOutlookAverage,ApprovesofCEOAverage,HelpfulAverage,CurrentEmployeeAverage,FullTimeAverage,TenureAverage,MarketCapAverage,MarketCapDelta,marketcapdummy,marketcapdummy_log,PF_changed,avg_previous_currentyear,avg_current,delta_currentprevious,realtivfemarketcapDelta,marketcapdummy_ersteBeobachtungb,gain_dummy,loss_dummy,replacement_dummy,change_dummy,event_gain_dummy,event_loss_dummy,event_replace_dummy,event_change_dummy,avg_current_valued_in_prioryear,delta_old,PhD,IvyLeague,AwardReceived,AuditCommittee,CompensationCommittee,NominationCommittee,GovernanceCommittee,FinanceRiskCommittee,Member,Chairman,ED,CEO,CFO,NumberofDirs,Nationality,NewAge,GenderDummy1M0F,SoleDirectorship,lnAge,gain_loss_replacement,new_chairman_dummy,Director Network Size
0,3791088892,6613355791,1999,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
1,3791088892,6613355791,2000,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
2,3791088892,6613355791,2001,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
3,3791088892,6613355791,2002,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217
4,3791088892,6613355791,2003,NaN,NaN,NaN,0,US1724061007,US1724062096,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,9.568189,-22.36,-8.857,NaN,NaN,NaN,NaN,NaN,NaN,18070.0,109.73,NaN,8551.0,NaN,34.66,60.87,NaN,NaN,NaN,3307.0,0.0,NaN,0.0,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,165354.0,NaN,0.0

In [6]:
#export
df_director_firm_year_merge_network.to_csv(r"C:\Users\hannes\Desktop\director-firm-years-complete.csv", sep=";", decimal='.', index=False)

## further controls 22 dec

In [ ]:
import pandas as pd
import numpy as np
df_latest = pd.read_csv(r"F:\Downloads\director-firm-years-with-glassdoor_18122021_clean.csv", sep=";", decimal='.', encoding = "ISO-8859-1")
df_latest.replace("#NV", np.nan, inplace=True)

df_board_controls = pd.read_excel(r"F:\Downloads\Further_Controls_22122021_merge.xlsx")

In [53]:
df_board_controls.head()

,DirectorID,FirmID,Year,ISIN,ISIN Alternative,Firm_board_attendance,Firm_board_size,Firm_board_compensation
0,3791088892,6613355791,1999,US1724061007,US1724062096,NaN,NaN,NaN
1,3791088892,6613355791,2000,US1724061007,US1724062096,NaN,NaN,NaN
2,3791088892,6613355791,2001,US1724061007,US1724062096,NaN,NaN,NaN
3,3791088892,6613355791,2002,US1724061007,US1724062096,NaN,NaN,NaN
4,3791088892,6613355791,2003,US1724061007,US1724062096,NaN,NaN,NaN


In [ ]:
pd.set_option('display.max_rows', 500)

df_latest.drop(columns=["PF_MarketCapAverage_NURTESTWEISEFÜRMICHDRINLÖSCHEICHMORGEN!!!!"], inplace=True)

for col in ["FirmID", "DirectorID", "FIRM_total_assets", "FIRM_beta", "PF_amort_of_def_charges_averages"]:
    df_latest[col] = df_latest[col].astype("float")
df_latest.dtypes

In [3]:
df_director_firm_year_merge_board_controls = df_latest.merge(df_board_controls, how="left", on=["DirectorID", "FirmID", "Year"])
df_director_firm_year_merge_board_controls.drop(columns=["ISIN_y", "ISIN Alternative_y"], inplace=True)
df_director_firm_year_merge_board_controls

,DirectorID,FirmID,Year,directorship_presence_in _PF,years_before_gain,years_after_loss,solo_observation_dummy,number_of_dirs_delta (DOPPELT? SIEHE SPALTE DK),gain_happened_overall,gain_happened_overall_dummy,...,FIRM_positive_outlook,FIRM_approves_of_CEO,FIRM_helpful,FIRM_current_employee,FIRM_full_time,FIRM_tenure,FIRM_rating_overall_count,Firm_board_attendance,Firm_board_size,Firm_board_compensation
0,3.791089e+09,6.613356e+09,1999,NaN,NaN,NaN,0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3.791089e+09,6.613356e+09,2000,NaN,NaN,NaN,0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.791089e+09,6.613356e+09,2001,NaN,NaN,NaN,0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3.791089e+09,6.613356e+09,2002,NaN,NaN,NaN,0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3.791089e+09,6.613356e+09,2003,NaN,NaN,NaN,0,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920950,4.782098e+09,2.286340e+10,2015,2.0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
920951,4.782098e+09,2.286340e+10,2016,1.0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
920952,4.782098e+09,2.286340e+10,2017,1.0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
920953,4.782098e+09,2.286340e+10,2018,1.0,NaN,NaN,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
#export
df_director_firm_year_merge_board_controls.to_csv(r"F:\Downloads\director-firm-years-with-board_controls_22-12.csv", sep=";", decimal='.', index=False)

## firm management departure & number of dirs (self-calculated, not based on boardex)

In [3]:
import pandas as pd
df_latest = pd.read_csv(r"F:\Downloads\director-firm-years-with-board_controls_23-12.csv", sep=";", decimal='.')
df_departure = pd.read_excel(r"F:\Downloads\management_departure.xlsx")
df_numberofdirs = pd.read_excel(r"C:\Users\Hannes\Desktop\number_of_dirs.xlsx")
df_latest

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,...,FIRM_board_attendance,FIRM_board_size,FIRM_board_compensation,DIR_Nationality,DIR_Age,DIR_GenderDummy1M0F,DIR_SoleDirectorship,DIR_lnAge,DIR_DirectorNetworkSize,FIXED_EFFECTS_CompositeIdentifier_DirectorFIRM
0,3.791089e+09,6.613356e+09,US1724061007,US1724062096,1999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19
1,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19
2,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19
3,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2002,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19
4,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2003,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,61.0,NaN,1.0,4.110874,217,3.790000e+19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
920950,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2015,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20
920951,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2016,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20
920952,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2017,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20
920953,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2018,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20


In [4]:
df_director_firm_year_merge_departure = df_latest.merge(df_departure, how="left", on=["DirectorID", "FirmID", "Year"])
df_director_firm_year_merge_departure

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,...,FIRM_board_size,FIRM_board_compensation,DIR_Nationality,DIR_Age,DIR_GenderDummy1M0F,DIR_SoleDirectorship,DIR_lnAge,DIR_DirectorNetworkSize,FIXED_EFFECTS_CompositeIdentifier_DirectorFIRM,FIRM_Management_Departure
0,3.791089e+09,6.613356e+09,US1724061007,US1724062096,1999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19,NaN
1,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19,NaN
2,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19,NaN
3,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2002,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19,NaN
4,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2003,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,61.0,NaN,1.0,4.110874,217,3.790000e+19,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929224,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2015,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20,NaN
929225,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2016,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20,NaN
929226,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2017,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20,NaN
929227,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2018,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20,NaN


In [5]:
df_director_firm_year_merge_numberofdirs = df_director_firm_year_merge_departure.merge(df_numberofdirs, how="left", on=["DirectorID", "Year"])
df_director_firm_year_merge_numberofdirs

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,...,FIRM_board_compensation,DIR_Nationality,DIR_Age,DIR_GenderDummy1M0F,DIR_SoleDirectorship,DIR_lnAge,DIR_DirectorNetworkSize,FIXED_EFFECTS_CompositeIdentifier_DirectorFIRM,FIRM_Management_Departure,PF_number_of_dirs_selfcalculated
0,3.791089e+09,6.613356e+09,US1724061007,US1724062096,1999,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19,NaN,NaN
1,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19,NaN,NaN
2,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2001,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19,NaN,NaN
3,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2002,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,217,3.790000e+19,NaN,NaN
4,3.791089e+09,6.613356e+09,US1724061007,US1724062096,2003,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,61.0,NaN,1.0,4.110874,217,3.790000e+19,NaN,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
929224,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2015,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20,NaN,NaN
929225,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2016,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20,NaN,NaN
929226,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2017,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20,NaN,NaN
929227,4.782098e+09,2.286340e+10,IE00BJ0X7W22,IE00BJ0X7W22,2018,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,109,4.780000e+20,NaN,NaN


In [6]:
#export
df_director_firm_year_merge_numberofdirs.to_csv(r"F:\Downloads\director-firm-years_with-numberofdirs_12-24.csv", sep=";", decimal='.', index=False)

## 3rd january merge with fixed IDs

In [1]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)

df_ego_marketcap = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Vars\Ego_Controls_03012022.xlsx")

df_firm_controls = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Vars\New_Board_Mgm_Controls_02012022_merge.xlsx")
df_firm_controls.drop_duplicates(ignore_index=True, inplace=True)

df_com = pd.read_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Vars\committees.csv", sep=";", decimal='.')

df_latest_correct_firmid_dirid = pd.read_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Director-Firm-Years\0103director-firm-years.csv", sep=";", decimal='.')
# print(df_latest_correct_firmid_dirid.iloc[:,149])
#convert object columns
df_latest_correct_firmid_dirid["PF_gain_loss_replace"].replace({"gain": 1, "gain doublecheck": 2, "loss": -1, "loss doublecheck": -2, "replace": 3}, inplace=True)
df_latest_correct_firmid_dirid["PF_gain_loss_replace"] = df_latest_correct_firmid_dirid["PF_gain_loss_replace"].astype("float")

df_latest_correct_firmid_dirid.drop(columns=["PF_litigation_expenses_average"], inplace=True) #numbers have wrong , in them

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     print(df_latest_correct_firmid_dirid.dtypes)

C:\Users\Hannes\AppData\Local\Programs\Python\Python37\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (5,75,150) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [43]:
df_ego_marketcap.head()

,DirectorID,Year,Ego_Market_Rep_Average,Ego_Market_Rep_Total
0,3791088892,2010,NaN,NaN
1,3762338623,1999,231822.0,231822.0
2,3762338623,2000,286054.0,286054.0
3,3762338623,2001,84152.0,84152.0
4,3762338623,2002,52670.5,105341.0


In [77]:
df_firm_controls

,Year,ISIN,FIRM_board_attendance,FIRM_board_size,FIRM_board_compensation,Management_Departure
0,1999,US1724061007,NaN,NaN,NaN,NaN
1,2000,US1724061007,NaN,NaN,NaN,NaN
2,2001,US1724061007,NaN,NaN,NaN,NaN
3,2002,US1724061007,NaN,NaN,NaN,NaN
4,2003,US1724061007,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
240084,2015,GB00B83VD954,NaN,NaN,NaN,NaN
240085,2016,GB00B83VD954,NaN,NaN,NaN,NaN
240086,2017,GB00B83VD954,NaN,NaN,NaN,NaN
240087,2018,GB00B83VD954,NaN,NaN,NaN,NaN


In [50]:
df_com.head()

,DirectorID,FirmID,Year,overlapping_committees_or_positions,InsideToOutsideRole,OutsideToInsideRole,Committee_Group_Nominating,Committee_Group_Compensation,Committee_Group_Audit,Committee_Group_Finance,Committee_Group_Governance,Committee_Group_Risk,Committee_Member,Committee_Chairman,Committee_OtherMember
0,216931,289744950,2004.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,216931,289744950,2005.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,216931,289744950,2006.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,216931,289744950,2007.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,216931,12728885777,2004.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
df_latest_egocap = df_latest_correct_firmid_dirid.merge(df_ego_marketcap, how="left", on=["DirectorID", "Year"])

#move columns
for index, column in enumerate(["Ego_Market_Rep_Average", "Ego_Market_Rep_Total"]):
    #cut column
    col = df_latest_egocap.pop(column)
    #insert column
    df_latest_egocap.insert(df_latest_egocap.columns.get_loc("PF_MarketCapTotal") - index, col.name, col)
    #rename
    df_latest_egocap.rename(columns={column: "PF_"+column}, inplace=True)

df_latest_egocap

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,PF_gain_loss_replacement_indicator,FIRM_presence_in _PF,years_before_gain,years_after_loss,FIRM_gain_prior_vs_event_dummy,FIRM_loss_prior_vs_event_dummy,solo_observation_dummy,PF_no_of_dirs_selfcalculated,PF_no_of_dirs_selfcalculated_delta,PF_number_of_dirs,PF_number_of_dirs_delta,gain_happened_max_changes,gain_happened_dummy,loss_happened_min_changes,loss_happened_dummy,PF_Ego_Market_Rep_Total,PF_Ego_Market_Rep_Average,PF_MarketCapTotal,PF_MarketCap_Average_woEventFirm,delta_FIRM_PF_mcap_woFirm_REVERSE,delta_FIRM_PF_mcap_w0_woFirm_REVERSE,delta_FIRM_PF_mcap_woFirm_dummy_REVERSE,delta_FIRM_PF_mcap__w0_woFirm_dummy_REVERSE,PF_MarketCapAverage,FIRM_market_cap_total,delta_FIRM_to_avg_PF_mcap_DELETE,delta_FIRM_to_avg_PF_mcap_dummy_DELETE,delta_FIRM_to_avg_PF_mcap_manual0,delta_FIRM_PF_mcap_manual0_REVERSE,delta_FIRM_PF_mcap_manual0_dummy,delta_FIRM_PF_mcap_manual0_dummy_REVERSE,delta_FIRM_PF_beta,delta_FIRM_PF_beta_REVERSE,delta_FIRM_PF_beta_dummy,delta_FIRM_PF_beta_dummy_REVERSE,delta_FIRM_PF_bmeetings,delta_FIRM_PF_bmeetings_REVERSE,delta_FIRM_PF_beetings_dummy,delta_FIRM_PF_bmeetings_dummy_REVERSE,delta_FIRM_PF_comp,delta_FIRM_PF_comp_REVERSE,delta_FIRM_PF_comp_dummy,delta_FIRM_PF_comp_dummy_REVERSE,PF_MarketCapAverage_delta_to_previous_value,PF_MarketCapAverage_delta_to_previous_value_relative,PF_MarketCapAverage_delta_to_previous_value_dummy_DELETE_includes_firstObs,PF_MarketCapAverage_delta_to_previous_value_dummy,PF_market_cap_delta_mlogit_dummy,PF_avg_previouspf_valued_at_currentyear,PF_avg_currentpf,PF_delta_previousvaluedatcurrent_minus_current,PF_delta_previousvaluedatcurrent_minus_current_realtivfe,PF_avg_currentpf_delta_to_previous_value,PF_event_gain_dummy_circularity,PF_event_loss_dummy_circularity,PF_event_replace_dummy_circularity,PF_event_change_dummy_circularity,avg_current_valued_in_prioryear,PF_beta_average,PF_beta_total,PF_vola_average,PF_vola_total,PF_tobinsQ_average,PF_ROA_average,PF_EPS_average,PF_board_meetings_average,PF_attendance_level,PF_independence_level_average,PF_accruals_change_average,PF_total_assets_average,PF_growth_assets_average,PF_impairment_intangibles_average,PF_cash_average,PF_impairment_goodwill_average,PF_debt_to_capital_average,PF_closely_held_shares_average,PF_amort_intang_goodwill_average,PF_amort_of_def_charges_averages,PF_amort_intang_average,PF_common_shares_out_average,PF_bribery_corruption_fraud_dummy,PF_research_exp_average,PF_early_auditor_resign_dummy,PF_managment_departure_dummy,PF_goodwill_writtenoff_average,PF_earnings_restatement_average,PF_governance_score_average,PF_board_size_average,PF_NED_board_size_average,PF_board_compensation_average,PF_GD_review_average,PF_GD_RatingBalanceAverage,PF_GD_RatingCultureOverallAverage,PF_GD_RatingCareerAverage,PF_GD_RatingCompAverage,PF_GD_RecommendsAverage,PF_GD_PositiveOutlookAverage,PF_GD_ApprovesofCEOAverage,PF_GD_HelpfulAverage,PF_GD_CurrentEmployeeAverage,PF_GD_FullTimeAverage,PF_GD_TenureAverage,PF_DIR_AuditCommittee_dummy,PF_DIR_CompensationCommittee_dummy,PF_DIR_NominationCommittee_dummy,PF_DIR_GovernanceCommittee_dummy,PF_DIR_FinanceRiskCommittee_dummy,PF_is_chairman_dummy,PF_DIR_ED,PF_DIR_CEO,PF_DIR_CFO,FIRM_beta,FIRM_vola,FIRM_board_meetings,FIRM_ESG_controversies_score,FIRM_board_independence,FIRM_total_assets,FIRM_growth_assets,FIRM_cash,FIRM_debt_to_capital,FIRM_closely_held_shares,FIRM_EPS,FIRM_ROA,FIRM_GD_rating_overall,FIRM_GD_rating_balance,FIRM_GD_rating_culture,FIRM_GD_rating_career,FIRM_GD_rating_comp,FIRM_GD_rating_mgmt,FIRM_GD_recommends,FIRM_GD_positive_outlook,FIRM_GD_approves_of_CEO,FIRM_GD_helpful,FIRM_GD_current_employee,FIRM_GD_full_time,FIRM_GD_tenure,FIRM_GD_rating_overall_count,FIRM_board_attendance,FIRM_board_size,FIRM_board_compensation,FIRM_Management_Departure,DIR_Nationality,DIR_PhD,DIR_IvyLeague,DIR_AwardsReceived,DIR_Age,DIR_GenderDummy1M0F,DIR_SoleDirectorship,DIR_lnA

In [3]:
#merge left (use first df as baseline. add columns from second df to baselind)
df_merge = df_latest_egocap.merge(df_com, how="left", on=["DirectorID", "FirmID", "Year"])
list(df_merge.columns.values)

#drop wrong outdated columns
com_list_drop = ['FIXED_EFFECTS_DirectorFIRM', 'DIR_FIRM_overlapping_committees_or_positions', 'DIR_FIRM_InsideToOutsideRole', 'DIR_FIRM_OutsideToInsideRole', 'DIR_FIRM_Committee_Group_Nominating', 'DIR_FIRM_Committee_Group_Compensation', 'DIR_FIRM_Committee_Group_Audit', 'DIR_FIRM_Committee_Group_Finance', 'DIR_FIRM_Committee_Group_Governance', 'DIR_FIRM_Committee_Group_Risk', 'DIR_FIRM_Committee_Member', 'DIR_FIRM_Committee_Chairman', 'DIR_FIRM_Committee_OtherMember']
df_merge.drop(columns=com_list_drop, inplace=True)

#add DIR_FIRM prefix to all columns starting after year column
com_list = ['overlapping_committees_or_positions', 'InsideToOutsideRole', 'OutsideToInsideRole', 'Committee_Group_Nominating', 'Committee_Group_Compensation', 'Committee_Group_Audit', 'Committee_Group_Finance', 'Committee_Group_Governance', 'Committee_Group_Risk', 'Committee_Member', 'Committee_Chairman', 'Committee_OtherMember']
df_merge.columns = ["DIR_FIRM_" + col if col in com_list else col for col in df_merge.columns]

df_merge

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,PF_gain_loss_replacement_indicator,FIRM_presence_in _PF,years_before_gain,years_after_loss,FIRM_gain_prior_vs_event_dummy,FIRM_loss_prior_vs_event_dummy,solo_observation_dummy,PF_no_of_dirs_selfcalculated,PF_no_of_dirs_selfcalculated_delta,PF_number_of_dirs,PF_number_of_dirs_delta,gain_happened_max_changes,gain_happened_dummy,loss_happened_min_changes,loss_happened_dummy,PF_Ego_Market_Rep_Total,PF_Ego_Market_Rep_Average,PF_MarketCapTotal,PF_MarketCap_Average_woEventFirm,delta_FIRM_PF_mcap_woFirm_REVERSE,delta_FIRM_PF_mcap_w0_woFirm_REVERSE,delta_FIRM_PF_mcap_woFirm_dummy_REVERSE,delta_FIRM_PF_mcap__w0_woFirm_dummy_REVERSE,PF_MarketCapAverage,FIRM_market_cap_total,delta_FIRM_to_avg_PF_mcap_DELETE,delta_FIRM_to_avg_PF_mcap_dummy_DELETE,delta_FIRM_to_avg_PF_mcap_manual0,delta_FIRM_PF_mcap_manual0_REVERSE,delta_FIRM_PF_mcap_manual0_dummy,delta_FIRM_PF_mcap_manual0_dummy_REVERSE,delta_FIRM_PF_beta,delta_FIRM_PF_beta_REVERSE,delta_FIRM_PF_beta_dummy,delta_FIRM_PF_beta_dummy_REVERSE,delta_FIRM_PF_bmeetings,delta_FIRM_PF_bmeetings_REVERSE,delta_FIRM_PF_beetings_dummy,delta_FIRM_PF_bmeetings_dummy_REVERSE,delta_FIRM_PF_comp,delta_FIRM_PF_comp_REVERSE,delta_FIRM_PF_comp_dummy,delta_FIRM_PF_comp_dummy_REVERSE,PF_MarketCapAverage_delta_to_previous_value,PF_MarketCapAverage_delta_to_previous_value_relative,PF_MarketCapAverage_delta_to_previous_value_dummy_DELETE_includes_firstObs,PF_MarketCapAverage_delta_to_previous_value_dummy,PF_market_cap_delta_mlogit_dummy,PF_avg_previouspf_valued_at_currentyear,PF_avg_currentpf,PF_delta_previousvaluedatcurrent_minus_current,PF_delta_previousvaluedatcurrent_minus_current_realtivfe,PF_avg_currentpf_delta_to_previous_value,PF_event_gain_dummy_circularity,PF_event_loss_dummy_circularity,PF_event_replace_dummy_circularity,PF_event_change_dummy_circularity,avg_current_valued_in_prioryear,PF_beta_average,PF_beta_total,PF_vola_average,PF_vola_total,PF_tobinsQ_average,PF_ROA_average,PF_EPS_average,PF_board_meetings_average,PF_attendance_level,PF_independence_level_average,PF_accruals_change_average,PF_total_assets_average,PF_growth_assets_average,PF_impairment_intangibles_average,PF_cash_average,PF_impairment_goodwill_average,PF_debt_to_capital_average,PF_closely_held_shares_average,PF_amort_intang_goodwill_average,PF_amort_of_def_charges_averages,PF_amort_intang_average,PF_common_shares_out_average,PF_bribery_corruption_fraud_dummy,PF_research_exp_average,PF_early_auditor_resign_dummy,PF_managment_departure_dummy,PF_goodwill_writtenoff_average,PF_earnings_restatement_average,PF_governance_score_average,PF_board_size_average,PF_NED_board_size_average,PF_board_compensation_average,PF_GD_review_average,PF_GD_RatingBalanceAverage,PF_GD_RatingCultureOverallAverage,PF_GD_RatingCareerAverage,PF_GD_RatingCompAverage,PF_GD_RecommendsAverage,PF_GD_PositiveOutlookAverage,PF_GD_ApprovesofCEOAverage,PF_GD_HelpfulAverage,PF_GD_CurrentEmployeeAverage,PF_GD_FullTimeAverage,PF_GD_TenureAverage,PF_DIR_AuditCommittee_dummy,PF_DIR_CompensationCommittee_dummy,PF_DIR_NominationCommittee_dummy,PF_DIR_GovernanceCommittee_dummy,PF_DIR_FinanceRiskCommittee_dummy,PF_is_chairman_dummy,PF_DIR_ED,PF_DIR_CEO,PF_DIR_CFO,FIRM_beta,FIRM_vola,FIRM_board_meetings,FIRM_ESG_controversies_score,FIRM_board_independence,FIRM_total_assets,FIRM_growth_assets,FIRM_cash,FIRM_debt_to_capital,FIRM_closely_held_shares,FIRM_EPS,FIRM_ROA,FIRM_GD_rating_overall,FIRM_GD_rating_balance,FIRM_GD_rating_culture,FIRM_GD_rating_career,FIRM_GD_rating_comp,FIRM_GD_rating_mgmt,FIRM_GD_recommends,FIRM_GD_positive_outlook,FIRM_GD_approves_of_CEO,FIRM_GD_helpful,FIRM_GD_current_employee,FIRM_GD_full_time,FIRM_GD_tenure,FIRM_GD_rating_overall_count,FIRM_board_attendance,FIRM_board_size,FIRM_board_compensation,FIRM_Management_Departure,DIR_Nationality,DIR_PhD,DIR_IvyLeague,DIR_AwardsReceived,DIR_Age,DIR_GenderDummy1M0F,DIR_SoleDirectorship,DIR_lnA

In [6]:
df_final = df_merge.merge(df_firm_controls, how="left", on=["ISIN", "Year"])
df_final.rename(columns={"Management_Departure": "FIRM_Management_Departure"}, inplace=True)
df_final["FE_Director_ISIN"] = df_final["DirectorID"].astype("str") + df_final["ISIN"]
df_final

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,PF_gain_loss_replacement_indicator,FIRM_presence_in _PF,years_before_gain,years_after_loss,FIRM_gain_prior_vs_event_dummy,FIRM_loss_prior_vs_event_dummy,solo_observation_dummy,PF_no_of_dirs_selfcalculated,PF_no_of_dirs_selfcalculated_delta,PF_number_of_dirs,PF_number_of_dirs_delta,gain_happened_max_changes,gain_happened_dummy,loss_happened_min_changes,loss_happened_dummy,PF_Ego_Market_Rep_Total,PF_Ego_Market_Rep_Average,PF_MarketCapTotal,PF_MarketCap_Average_woEventFirm,delta_FIRM_PF_mcap_woFirm_REVERSE,delta_FIRM_PF_mcap_w0_woFirm_REVERSE,delta_FIRM_PF_mcap_woFirm_dummy_REVERSE,delta_FIRM_PF_mcap__w0_woFirm_dummy_REVERSE,PF_MarketCapAverage,FIRM_market_cap_total,delta_FIRM_to_avg_PF_mcap_DELETE,delta_FIRM_to_avg_PF_mcap_dummy_DELETE,delta_FIRM_to_avg_PF_mcap_manual0,delta_FIRM_PF_mcap_manual0_REVERSE,delta_FIRM_PF_mcap_manual0_dummy,delta_FIRM_PF_mcap_manual0_dummy_REVERSE,delta_FIRM_PF_beta,delta_FIRM_PF_beta_REVERSE,delta_FIRM_PF_beta_dummy,delta_FIRM_PF_beta_dummy_REVERSE,delta_FIRM_PF_bmeetings,delta_FIRM_PF_bmeetings_REVERSE,delta_FIRM_PF_beetings_dummy,delta_FIRM_PF_bmeetings_dummy_REVERSE,delta_FIRM_PF_comp,delta_FIRM_PF_comp_REVERSE,delta_FIRM_PF_comp_dummy,delta_FIRM_PF_comp_dummy_REVERSE,PF_MarketCapAverage_delta_to_previous_value,PF_MarketCapAverage_delta_to_previous_value_relative,PF_MarketCapAverage_delta_to_previous_value_dummy_DELETE_includes_firstObs,PF_MarketCapAverage_delta_to_previous_value_dummy,PF_market_cap_delta_mlogit_dummy,PF_avg_previouspf_valued_at_currentyear,PF_avg_currentpf,PF_delta_previousvaluedatcurrent_minus_current,PF_delta_previousvaluedatcurrent_minus_current_realtivfe,PF_avg_currentpf_delta_to_previous_value,PF_event_gain_dummy_circularity,PF_event_loss_dummy_circularity,PF_event_replace_dummy_circularity,PF_event_change_dummy_circularity,avg_current_valued_in_prioryear,PF_beta_average,PF_beta_total,PF_vola_average,PF_vola_total,PF_tobinsQ_average,PF_ROA_average,PF_EPS_average,PF_board_meetings_average,PF_attendance_level,PF_independence_level_average,PF_accruals_change_average,PF_total_assets_average,PF_growth_assets_average,PF_impairment_intangibles_average,PF_cash_average,PF_impairment_goodwill_average,PF_debt_to_capital_average,PF_closely_held_shares_average,PF_amort_intang_goodwill_average,PF_amort_of_def_charges_averages,PF_amort_intang_average,PF_common_shares_out_average,PF_bribery_corruption_fraud_dummy,PF_research_exp_average,PF_early_auditor_resign_dummy,PF_managment_departure_dummy,PF_goodwill_writtenoff_average,PF_earnings_restatement_average,PF_governance_score_average,PF_board_size_average,PF_NED_board_size_average,PF_board_compensation_average,PF_GD_review_average,PF_GD_RatingBalanceAverage,PF_GD_RatingCultureOverallAverage,PF_GD_RatingCareerAverage,PF_GD_RatingCompAverage,PF_GD_RecommendsAverage,PF_GD_PositiveOutlookAverage,PF_GD_ApprovesofCEOAverage,PF_GD_HelpfulAverage,PF_GD_CurrentEmployeeAverage,PF_GD_FullTimeAverage,PF_GD_TenureAverage,PF_DIR_AuditCommittee_dummy,PF_DIR_CompensationCommittee_dummy,PF_DIR_NominationCommittee_dummy,PF_DIR_GovernanceCommittee_dummy,PF_DIR_FinanceRiskCommittee_dummy,PF_is_chairman_dummy,PF_DIR_ED,PF_DIR_CEO,PF_DIR_CFO,FIRM_beta,FIRM_vola,FIRM_board_meetings,FIRM_ESG_controversies_score,FIRM_board_independence,FIRM_total_assets,FIRM_growth_assets,FIRM_cash,FIRM_debt_to_capital,FIRM_closely_held_shares,FIRM_EPS,FIRM_ROA,FIRM_GD_rating_overall,FIRM_GD_rating_balance,FIRM_GD_rating_culture,FIRM_GD_rating_career,FIRM_GD_rating_comp,FIRM_GD_rating_mgmt,FIRM_GD_recommends,FIRM_GD_positive_outlook,FIRM_GD_approves_of_CEO,FIRM_GD_helpful,FIRM_GD_current_employee,FIRM_GD_full_time,FIRM_GD_tenure,FIRM_GD_rating_overall_count,FIRM_board_attendance_x,FIRM_board_size_x,FIRM_board_compensation_x,FIRM_Management_Departure,DIR_Nationality,DIR_PhD,DIR_IvyLeague,DIR_AwardsReceived,DIR_Age,DIR_GenderDummy1M0F,DIR_SoleDirectorship,D

In [7]:
#export
df_final.to_csv(r"F:\Downloads\0103director-firm-years_v2.csv", sep=";", decimal='.', index=False)

### industry codes

In [8]:
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)

df_industry = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Vars\SIC.xlsx")
# df_final = pd.read_csv(r"F:\Downloads\0103director-firm-years_v2.csv", sep=";", decimal='.')

df_industry.head()

,ISIN,FIRM_SIC,FIRM_SIC1,FIRM_SIC2
0,US6369183024,6324.0,6.0,63.0
1,US46333X1081,2834.0,2.0,28.0
2,US22304C1009,742.0,7.0,74.0
3,US4945801037,8059.0,8.0,80.0
4,US44667X2080,NaN,NaN,NaN


In [9]:
df_final = df_final.merge(df_industry, how="left", on=["ISIN"])
df_final

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,PF_gain_loss_replacement_indicator,FIRM_presence_in _PF,years_before_gain,years_after_loss,FIRM_gain_prior_vs_event_dummy,FIRM_loss_prior_vs_event_dummy,solo_observation_dummy,PF_no_of_dirs_selfcalculated,PF_no_of_dirs_selfcalculated_delta,PF_number_of_dirs,PF_number_of_dirs_delta,gain_happened_max_changes,gain_happened_dummy,loss_happened_min_changes,loss_happened_dummy,PF_Ego_Market_Rep_Total,PF_Ego_Market_Rep_Average,PF_MarketCapTotal,PF_MarketCap_Average_woEventFirm,delta_FIRM_PF_mcap_woFirm_REVERSE,delta_FIRM_PF_mcap_w0_woFirm_REVERSE,delta_FIRM_PF_mcap_woFirm_dummy_REVERSE,delta_FIRM_PF_mcap__w0_woFirm_dummy_REVERSE,PF_MarketCapAverage,FIRM_market_cap_total,delta_FIRM_to_avg_PF_mcap_DELETE,delta_FIRM_to_avg_PF_mcap_dummy_DELETE,delta_FIRM_to_avg_PF_mcap_manual0,delta_FIRM_PF_mcap_manual0_REVERSE,delta_FIRM_PF_mcap_manual0_dummy,delta_FIRM_PF_mcap_manual0_dummy_REVERSE,delta_FIRM_PF_beta,delta_FIRM_PF_beta_REVERSE,delta_FIRM_PF_beta_dummy,delta_FIRM_PF_beta_dummy_REVERSE,delta_FIRM_PF_bmeetings,delta_FIRM_PF_bmeetings_REVERSE,delta_FIRM_PF_beetings_dummy,delta_FIRM_PF_bmeetings_dummy_REVERSE,delta_FIRM_PF_comp,delta_FIRM_PF_comp_REVERSE,delta_FIRM_PF_comp_dummy,delta_FIRM_PF_comp_dummy_REVERSE,PF_MarketCapAverage_delta_to_previous_value,PF_MarketCapAverage_delta_to_previous_value_relative,PF_MarketCapAverage_delta_to_previous_value_dummy_DELETE_includes_firstObs,PF_MarketCapAverage_delta_to_previous_value_dummy,PF_market_cap_delta_mlogit_dummy,PF_avg_previouspf_valued_at_currentyear,PF_avg_currentpf,PF_delta_previousvaluedatcurrent_minus_current,PF_delta_previousvaluedatcurrent_minus_current_realtivfe,PF_avg_currentpf_delta_to_previous_value,PF_event_gain_dummy_circularity,PF_event_loss_dummy_circularity,PF_event_replace_dummy_circularity,PF_event_change_dummy_circularity,avg_current_valued_in_prioryear,PF_beta_average,PF_beta_total,PF_vola_average,PF_vola_total,PF_tobinsQ_average,PF_ROA_average,PF_EPS_average,PF_board_meetings_average,PF_attendance_level,PF_independence_level_average,PF_accruals_change_average,PF_total_assets_average,PF_growth_assets_average,PF_impairment_intangibles_average,PF_cash_average,PF_impairment_goodwill_average,PF_debt_to_capital_average,PF_closely_held_shares_average,PF_amort_intang_goodwill_average,PF_amort_of_def_charges_averages,PF_amort_intang_average,PF_common_shares_out_average,PF_bribery_corruption_fraud_dummy,PF_research_exp_average,PF_early_auditor_resign_dummy,PF_managment_departure_dummy,PF_goodwill_writtenoff_average,PF_earnings_restatement_average,PF_governance_score_average,PF_board_size_average,PF_NED_board_size_average,PF_board_compensation_average,PF_GD_review_average,PF_GD_RatingBalanceAverage,PF_GD_RatingCultureOverallAverage,PF_GD_RatingCareerAverage,PF_GD_RatingCompAverage,PF_GD_RecommendsAverage,PF_GD_PositiveOutlookAverage,PF_GD_ApprovesofCEOAverage,PF_GD_HelpfulAverage,PF_GD_CurrentEmployeeAverage,PF_GD_FullTimeAverage,PF_GD_TenureAverage,PF_DIR_AuditCommittee_dummy,PF_DIR_CompensationCommittee_dummy,PF_DIR_NominationCommittee_dummy,PF_DIR_GovernanceCommittee_dummy,PF_DIR_FinanceRiskCommittee_dummy,PF_is_chairman_dummy,PF_DIR_ED,PF_DIR_CEO,PF_DIR_CFO,FIRM_beta,FIRM_vola,FIRM_board_meetings,FIRM_ESG_controversies_score,FIRM_board_independence,FIRM_total_assets,FIRM_growth_assets,FIRM_cash,FIRM_debt_to_capital,FIRM_closely_held_shares,FIRM_EPS,FIRM_ROA,FIRM_GD_rating_overall,FIRM_GD_rating_balance,FIRM_GD_rating_culture,FIRM_GD_rating_career,FIRM_GD_rating_comp,FIRM_GD_rating_mgmt,FIRM_GD_recommends,FIRM_GD_positive_outlook,FIRM_GD_approves_of_CEO,FIRM_GD_helpful,FIRM_GD_current_employee,FIRM_GD_full_time,FIRM_GD_tenure,FIRM_GD_rating_overall_count,FIRM_board_attendance_x,FIRM_board_size_x,FIRM_board_compensation_x,FIRM_Management_Departure,DIR_Nationality,DIR_PhD,DIR_IvyLeague,DIR_AwardsReceived,DIR_Age,DIR_GenderDummy1M0F,DIR_SoleDirectorship,D

In [91]:
df_final.to_csv(r"F:\Downloads\0103director-firm-years_v3.csv", sep=";", decimal='.', index=False)

### esg score

In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

df = pd.read_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Director-Firm-Years\0106director-firm-years_delta.csv", sep=";", decimal='.', dtype={"DIR_Nationality": str}) #DtypeWarning: Columns (195) have mixed types.Specify dtype option on import 
df_esg = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Vars\Merge_07012022.xlsx", sheet_name="Aus Ausgangsdatei")

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_Ego_Market_Rep_Avg_delta_prev,PF_Ego_Market_Rep_Avg_delta_prev_rel,PF_Ego_Market_Rep_Avg_delta_prev_rel_dummy,FIRM_ESG_Score
0,216931,289744950,US6369183024,US6369183024,1999,NaN,NaN,NaN,NaN
1,216931,289744950,US6369183024,US6369183024,2000,NaN,NaN,NaN,NaN
2,216931,289744950,US6369183024,US6369183024,2001,NaN,NaN,NaN,NaN
3,216931,289744950,US6369183024,US6369183024,2002,NaN,NaN,NaN,NaN
4,216931,289744950,US6369183024,US6369183024,2003,NaN,NaN,NaN,NaN


In [22]:
#remove firmid and isin alternative
df_esg.columns.to_list()
df_esg = df_esg[['DirectorID','ISIN','Year','PF_Ego_Market_Rep_Avg_delta_prev','PF_Ego_Market_Rep_Avg_delta_prev_rel','PF_Ego_Market_Rep_Avg_delta_prev_rel_dummy','FIRM_ESG_Score']]

df_esg.head()

,DirectorID,ISIN,Year,PF_Ego_Market_Rep_Avg_delta_prev,PF_Ego_Market_Rep_Avg_delta_prev_rel,PF_Ego_Market_Rep_Avg_delta_prev_rel_dummy,FIRM_ESG_Score
0,216931,US6369183024,1999,NaN,NaN,NaN,NaN
1,216931,US6369183024,2000,NaN,NaN,NaN,NaN
2,216931,US6369183024,2001,NaN,NaN,NaN,NaN
3,216931,US6369183024,2002,NaN,NaN,NaN,NaN
4,216931,US6369183024,2003,NaN,NaN,NaN,NaN


In [29]:
df_merged = df.merge(df_esg, how="left", on=["DirectorID", "ISIN", "Year"])

#move columns
col = df_merged.pop("FIRM_ESG_Score")
df_merged.insert(df_merged.columns.get_loc("FIRM_ESG_controversies_score") + 1, col.name, col)

gainer_cols = ["PF_Ego_Market_Rep_Avg_delta_prev", "PF_Ego_Market_Rep_Avg_delta_prev_rel", "PF_Ego_Market_Rep_Avg_delta_prev_rel_dummy"]
for index, column in enumerate(gainer_cols, start=1):
    #cut column
    col = df_merged.pop(column)
    #insert column
    df_merged.insert(df_merged.columns.get_loc("PF_MarketCap_withoutEvent_ego") + index, col.name, col)

#df has duplicate dir-isin-year entries for whatever reason
#display duplicates. should be empty
# df_merged["check"] = df_merged.duplicated(subset=["DirectorID", "ISIN", "Year"], keep=False) #False : Mark all duplicates as True
# df_duplicate_rows = df_merged[df_merged["check"] == True].sort_values(by=["DirectorID", "ISIN", "Year"], ascending=[True,True, True], ignore_index=True)
# df_duplicate_rows
# df_merged.drop(columns=["check"], inplace=True)

#remove duplicates
df_merged.drop_duplicates(subset=["DirectorID", "ISIN", "Year"], keep="first", inplace=True) #only keep first entry of duplicates

#convert string values to nan
df_merged["FIRM_ESG_Score"][df_merged["FIRM_ESG_Score"].str.contains('$', na=False)] = np.nan #esg score has string values (containing $$ missing) -> convert to nan

df_merged

,DirectorID,FirmID,ISIN,ISIN_alternative,Year,PF_active,PF_max_gains_per_director,PF_gain_loss_replace,PF_gain_dummy,PF_loss_dummy,PF_replace_dummy,PF_change_dummy,PF_gain_loss_replacement_indicator,PF_gainer,PF_looser,PF_changer,FIRM_presence_in_PF,years_before_gain,years_after_loss,FIRM_gain_prior_vs_event_dummy,FIRM_loss_prior_vs_event_dummy,solo_observation_dummy,PF_no_of_dirs_selfcalculated,PF_no_of_dirs_selfcalculated_delta,PF_number_of_dirs,PF_number_of_dirs_delta,PF_no_of_dirs_selfcalc_ego_delta,PF_no_of_dirs_selfcalc_ego,gain_happened_max_changes,gain_happened_dummy,loss_happened_min_changes,loss_happened_dummy,PF_Ego_Market_Rep_Average,PF_MarketCap_Total_ego,PF_MarketCap_withoutEvent_ego,PF_Ego_Market_Rep_Avg_delta_prev,PF_Ego_Market_Rep_Avg_delta_prev_rel,PF_Ego_Market_Rep_Avg_delta_prev_rel_dummy,delta_MarketCap_ego,delta_MarketCap_ego_dummy,PF_MarketCapTotal,PF_MarketCap_Average_woEventFirm,delta_FIRM_PF_mcap_woFirm_REVERSE,delta_FIRM_PF_mcap_w0_woFirm_REVERSE,delta_FIRM_PF_mcap_woFirm_dummy_REVERSE,delta_FIRM_PF_mcap__w0_woFirm_dummy_REVERSE,PF_MarketCapAverage,FIRM_market_cap_total,delta_FIRM_to_avg_PF_mcap_DELETE,delta_FIRM_to_avg_PF_mcap_dummy_DELETE,delta_FIRM_to_avg_PF_mcap_manual0,delta_FIRM_PF_mcap_manual0_REVERSE,delta_FIRM_PF_mcap_manual0_dummy,delta_FIRM_PF_mcap_manual0_dummy_REVERSE,delta_FIRM_PF_beta,delta_FIRM_PF_beta_REVERSE,delta_FIRM_PF_beta_dummy,delta_FIRM_PF_beta_dummy_REVERSE,delta_FIRM_PF_bmeetings,delta_FIRM_PF_bmeetings_REVERSE,delta_FIRM_PF_beetings_dummy,delta_FIRM_PF_bmeetings_dummy_REVERSE,delta_FIRM_PF_comp,delta_FIRM_PF_comp_REVERSE,delta_FIRM_PF_comp_dummy,delta_FIRM_PF_comp_dummy_REVERSE,delta_FIRM_PF_GD_rating_overall,PF_MarketCapAverage_delta_to_previous_value,PF_MarketCapAverage_delta_to_previous_value_relative,PF_MarketCapAverage_delta_to_previous_value_dummy_DELETE_includes_firstObs,PF_MarketCapAverage_delta_to_previous_value_dummy,PF_market_cap_delta_mlogit_dummy,PF_avg_previouspf_valued_at_currentyear,PF_avg_currentpf,PF_delta_previousvaluedatcurrent_minus_current,PF_delta_previousvaluedatcurrent_minus_current_realtivfe,PF_avg_currentpf_delta_to_previous_value,PF_event_gain_dummy_circularity,PF_event_loss_dummy_circularity,PF_event_replace_dummy_circularity,PF_event_change_dummy_circularity,PF_avg_current_valued_in_prioryear,PF_beta_average,PF_beta_total,PF_vola_average,PF_vola_total,PF_tobinsQ_average,PF_ROA_average,PF_EPS_average,PF_board_meetings_average,PF_attendance_level,PF_independence_level_average,PF_accruals_change_average,PF_total_assets_average,PF_growth_assets_average,PF_impairment_intangibles_average,PF_cash_average,PF_impairment_goodwill_average,PF_debt_to_capital_average,PF_closely_held_shares_average,PF_amort_intang_goodwill_average,PF_amort_of_def_charges_averages,PF_amort_intang_average,PF_common_shares_out_average,PF_bribery_corruption_fraud_dummy,PF_research_exp_average,PF_early_auditor_resign_dummy,PF_managment_departure_dummy,PF_goodwill_writtenoff_average,PF_earnings_restatement_average,PF_governance_score_average,PF_board_size_average,PF_NED_board_size_average,PF_board_compensation_average,PF_is_chairman_dummy,PF_GD_rating_overall_average_withEventFirm,PF_GD_rating_balance_average_withEventFirm,PF_GD_rating_culture_average_withEventFirm,PF_GD_rating_career_average_withEventFirm,PF_GD_rating_comp_average_withEventFirm,PF_GD_rating_mgmt_average_withEventFirm,PF_GD_recommends_average_withEventFirm,PF_GD_positive_outlook_average_withEventFirm,PF_GD_approves_of_CEO_average_withEventFirm,PF_GD_helpful_average_withEventFirm,PF_GD_current_employee_average_withEventFirm,PF_GD_full_time_average_withEventFirm,PF_GD_tenure_average_withEventFirm,PF_GD_rating_overall_count_average_withEventFirm,PF_GD_rating_overall_sum_withEventFirm,PF_GD_rating_balance_sum_withEventFirm,PF_GD_rating_culture_sum_withEventFirm,PF_GD_rating_career_sum_withEventFirm,PF_GD_rating_comp_sum_withEventFirm,PF_GD_rating_mgmt_sum_withEventFirm,PF_GD_recommends_sum_withEventFirm,PF_GD_positiv

In [44]:
df_merged.to_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Director-Firm-Years\0109director-firm-years_delta.csv", sep=";", decimal='.', index=False)

### recreate age & DIR_gain_happened & dir-firm tenure [10 Jan]

In [14]:
df = pd.read_excel(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\Vars\dir_gain.xlsx")
df

,DirectorID,ISIN,Year,FIRM_presence_in_PF
0,216931,US6369183024,1999,NaN
1,216931,US6369183024,2000,NaN
2,216931,US6369183024,2001,NaN
3,216931,US6369183024,2002,NaN
4,216931,US6369183024,2003,NaN
...,...,...,...,...
916288,206877812921,KYG2345N1025,2015,NaN
916289,206877812921,KYG2345N1025,2016,NaN
916290,206877812921,KYG2345N1025,2017,2.0
916291,206877812921,KYG2345N1025,2018,1.0


#### calc tenure

In [92]:
df_active = df[df["FIRM_presence_in_PF"].isna() == False].drop(columns=["FIRM_presence_in_PF"])

#count up duplicate year rows
# https://stackoverflow.com/questions/59334873/i-want-to-count-the-occurrence-of-duplicate-values-in-a-column-in-a-dataframe-an
df_active['DIR_FIRM_tenure'] = df_active.groupby(['DirectorID','ISIN'], sort=False).cumcount()

df_active

,DirectorID,ISIN,Year,DIR_FIRM_tenure
5,216931,US6369183024,2004,0
6,216931,US6369183024,2005,1
7,216931,US6369183024,2006,2
8,216931,US6369183024,2007,3
32,216931,US46333X1081,2010,0
...,...,...,...,...
916270,206515512893,VGG273581030,2018,0
916271,206515512893,VGG273581030,2019,1
916290,206877812921,KYG2345N1025,2017,0
916291,206877812921,KYG2345N1025,2018,1


In [96]:
df_export = df.merge(df_active, how="left", on=["DirectorID", "ISIN", "Year"])
df_export
# df_export.to_csv(r"D:\Docs\Dokumente\WMA\Projects\5 Directors\Data\vars\tenure.csv", sep=";", decimal='.', index=False)

,DirectorID,ISIN,Year,FIRM_presence_in_PF,DIR_FIRM_tenure
0,216931,US6369183024,1999,NaN,NaN
1,216931,US6369183024,2000,NaN,NaN
2,216931,US6369183024,2001,NaN,NaN
3,216931,US6369183024,2002,NaN,NaN
4,216931,US6369183024,2003,NaN,NaN
...,...,...,...,...,...
916288,206877812921,KYG2345N1025,2015,NaN,NaN
916289,206877812921,KYG2345N1025,2016,NaN,NaN
916290,206877812921,KYG2345N1025,2017,2.0,0.0
916291,206877812921,KYG2345N1025,2018,1.0,1.0


#### calc gain/loss happened [FIRM_presence_in_PF outdated]

In [70]:
df_gain = df.groupby(["DirectorID", "Year"])["FIRM_presence_in_PF"].max().reset_index(name="DIR_gain_happened") #set to 2 if at least one firm was gained by dir during specific year
df_gain["DIR_gain_happened"] = np.where(df_gain["DIR_gain_happened"] == 2, 1, 0) #set to 1 if at least one firm was gained by dir in specific year


df_loss = df.groupby(["DirectorID", "Year"])["FIRM_presence_in_PF"].min().reset_index(name="DIR_loss_happened") #set to 2 if at least one firm was lossed by dir during specific year
df_loss["DIR_loss_happened"] = np.where(df_loss["DIR_loss_happened"] == -1, 1, 0) #set to 1 if at least one firm was loss by dir in specific year
df_loss[df_loss["DIR_loss_happened"] > 0]

,DirectorID,Year,DIR_loss_happened
8,216931,2007,1
15,216931,2014,1
40,611920,2018,1
44,722600,2001,1
71,722610,2007,1
...,...,...,...
479323,204779612774,2018,1
479617,205774712844,2018,1
479672,205968512858,2010,1
479827,206488112893,2018,1


In [56]:
df_gain = df.groupby(["DirectorID", "Year"], as_index=False).agg({'FIRM_presence_in_PF': ["max", "min"]}) 
grouped_col_names = [f"{x}_{y}" if "FIRM_" in x else x for x, y in df_gain.columns.to_flat_index()]
df_gain.columns = grouped_col_names

#observations where dir gained and lost one dir during same year
df_gain["DIR_replace_happened"] = np.where((df_gain["FIRM_presence_in_PF_max"] == 2) & (df_gain["FIRM_presence_in_PF_min"] == -1), 1, 0)

df_gain["DIR_gain_happened"] = np.where(df_gain["FIRM_presence_in_PF_max"] == 2, 1, 0) #set to 1 if at least one firm was gained by dir in specific year
df_gain["DIR_loss_happened"] = np.where(df_gain["FIRM_presence_in_PF_min"] == -1, 1, 0) #set to 1 if at least one firm was loss by dir in specific year

df_gain[df_gain["DIR_loss_happened"] == 1]

,DirectorID,Year,FIRM_presence_in_PF_max,FIRM_presence_in_PF_min,DIR_replace_happened,DIR_gain_happened,DIR_loss_happened
8,216931,2007,-1.0,-1.0,0,0,1
15,216931,2014,-1.0,-1.0,0,0,1
40,611920,2018,-1.0,-1.0,0,0,1
44,722600,2001,-1.0,-1.0,0,0,1
71,722610,2007,1.0,-1.0,0,0,1
...,...,...,...,...,...,...,...
479323,204779612774,2018,-1.0,-1.0,0,0,1
479617,205774712844,2018,-1.0,-1.0,0,0,1
479672,205968512858,2010,-1.0,-1.0,0,0,1
479827,206488112893,2018,2.0,-1.0,1,1,1


In [61]:
df_gain.drop(columns=["FIRM_presence_in_PF_max", "FIRM_presence_in_PF_min"], inplace=True)

df_merge = df_gain.merge(df_age, how="left", on=["DirectorID"]).fillna(np.nan)
df_merge["DIR_Age_complete"] = np.where(df_merge["DOB"].isna() == False, df_merge["Year"] - df_merge["DOB"], np.nan)
df_merge.drop(columns=["DOB"], inplace=True)
df_merge

,DirectorID,Year,DIR_replace_happened,DIR_gain_happened,DIR_loss_happened,DIR_Age_complete
0,216931,1999,0,0,0,48
1,216931,2000,0,0,0,49
2,216931,2001,0,0,0,50
3,216931,2002,0,0,0,51
4,216931,2003,0,0,0,52
...,...,...,...,...,...,...
480076,207000312928,2015,0,0,0,NaN
480077,207000312928,2016,0,0,0,NaN
480078,207000312928,2017,0,0,0,NaN
480079,207000312928,2018,0,1,0,NaN


In [66]:
#display duplicates. should be empty
df_merge["check"] = df_merge.duplicated(subset=["DirectorID", "Year"], keep=False) #False : Mark all duplicates as True
df_duplicate_rows = df_merge[df_merge["check"] == True].sort_values(by=["DirectorID", "Year"], ascending=[True,True], ignore_index=True)
df_duplicate_rows
# df_merge.drop(columns=["check"], inplace=True)

In [74]:
df_latest = df[["DirectorID", "Year"]]
df_final = df_latest.merge(df_merge, how="left", on=["DirectorID", "Year"])
df_final

,DirectorID,Year,DIR_replace_happened,DIR_gain_happened,DIR_loss_happened,DIR_Age_complete
0,216931,1999,0,0,0,48
1,216931,2000,0,0,0,49
2,216931,2001,0,0,0,50
3,216931,2002,0,0,0,51
4,216931,2003,0,0,0,52
...,...,...,...,...,...,...
916288,206877812921,2015,0,0,0,47
916289,206877812921,2016,0,0,0,48
916290,206877812921,2017,0,1,0,49
916291,206877812921,2018,0,0,0,50
